# Pachyderm Get Started Demo Sample: Data Versioning
Copyright (C) 2021 OS-Climate

This sample shows:
* How to ingest data from an external source with data versioning in place
* How to connect to pachyderm service, create a new repository and use versioning for data input and output
* 


Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and parquet file i/o
%pip install pandas

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools

# Python client for Pachyderm
%pip install python_pachyderm

In [2]:
# Report exceptions only in cells output
%xmode Minimal

Exception reporting mode: Minimal


In [3]:
from dotenv import dotenv_values, load_dotenv
import os
import pathlib
import boto3
from io import BytesIO
import pandas as pd
import trino
from sqlalchemy.engine import create_engine
import osc_ingest_trino as osc
import python_pachyderm

Define Environment and Execution Variables

In [4]:
# Define execution variables for loading to Iceberg / Trino
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'sandbox'
ingest_table = 'trino_iceberg_demo'

In [5]:
# Load environment variables from credentials.env
osc.load_credentials_dotenv()

Create an S3 resource for the bucket holding source data, then retrieve the relevant bucket and list objects

In [6]:
s3_resource = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)

In [7]:
bucket = s3_resource.Bucket(os.environ['S3_LANDING_BUCKET'])

# list all objects
for obj in bucket.objects.filter(Prefix='WRI/global_power_plant_database_v_1_3/'):
    print(obj.key, obj.last_modified)

WRI/global_power_plant_database_v_1_3/A_Global_Database_of_Power_Plants.pdf 2021-09-28 15:48:32+00:00
WRI/global_power_plant_database_v_1_3/Estimating_Power_Plant_Generation_in_the_Global_Power_Plant_Database.pdf 2021-09-28 15:48:26+00:00
WRI/global_power_plant_database_v_1_3/README.txt 2021-09-28 15:48:31+00:00
WRI/global_power_plant_database_v_1_3/RELEASE_NOTES.txt 2021-09-28 15:48:31+00:00
WRI/global_power_plant_database_v_1_3/global_power_plant_database.csv 2021-09-28 15:48:29+00:00


Load the CSV FILE for GPPD and convert it into a dataframe for processing

In [8]:
gppd_csv = s3_resource.Object(os.environ['S3_LANDING_BUCKET'], 'WRI/global_power_plant_database_v_1_3/global_power_plant_database.csv')
gppd_csv_file = BytesIO(gppd_csv.get()['Body'].read())

In [9]:
df_gppd = pd.read_csv(gppd_csv_file, dtype={'other_fuel3': str}).convert_dtypes()
print(df_gppd.info(verbose=True))
df_gppd.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34936 entries, 0 to 34935
Data columns (total 36 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   country                         34936 non-null  string 
 1   country_long                    34936 non-null  string 
 2   name                            34936 non-null  string 
 3   gppd_idnr                       34936 non-null  string 
 4   capacity_mw                     34936 non-null  Float64
 5   latitude                        34936 non-null  Float64
 6   longitude                       34936 non-null  Float64
 7   primary_fuel                    34936 non-null  string 
 8   other_fuel1                     1944 non-null   string 
 9   other_fuel2                     276 non-null    string 
 10  other_fuel3                     92 non-null     string 
 11  commissioning_year              17447 non-null  Float64
 12  owner                           

,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,primary_fuel,other_fuel1,other_fuel2,...,estimated_generation_gwh_2013,estimated_generation_gwh_2014,estimated_generation_gwh_2015,estimated_generation_gwh_2016,estimated_generation_gwh_2017,estimated_generation_note_2013,estimated_generation_note_2014,estimated_generation_note_2015,estimated_generation_note_2016,estimated_generation_note_2017
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,GEODB0040538,33.0,32.322,65.119,Hydro,<NA>,<NA>,...,123.77,162.9,97.39,137.76,119.5,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1
1,AFG,Afghanistan,Kandahar DOG,WKS0070144,10.0,31.67,65.795,Solar,<NA>,<NA>,...,18.43,17.48,18.25,17.7,18.29,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE
2,AFG,Afghanistan,Kandahar JOL,WKS0071196,10.0,31.623,65.792,Solar,<NA>,<NA>,...,18.64,17.58,19.1,17.62,18.72,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE
3,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,GEODB0040541,66.0,34.556,69.4787,Hydro,<NA>,<NA>,...,225.06,203.55,146.9,230.18,174.91,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1
4,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,GEODB0040534,100.0,34.641,69.717,Hydro,<NA>,<NA>,...,406.16,357.22,270.99,395.38,350.8,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1


In [10]:
# Create a pachyderm client by manually setting up host and port
client = python_pachyderm.Client(os.environ['PACH_ENDPOINT'], os.environ['PACH_PORT'])

In [11]:
# Create a pachyderm repo called `test`
client.create_repo("gppd")

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "repo gppd already exists"
	debug_error_string = "{"created":"@1648994242.088443590","description":"Error received from peer ipv4:172.30.83.114:30650","file":"src/core/lib/surface/call.cc","file_line":905,"grpc_message":"repo gppd already exists","grpc_status":2}"
>

In [12]:
# Display list of repos
print(list(client.list_repo()))

[repo {
  name: "gppd"
  type: "user"
}
created {
  seconds: 1648985432
  nanos: 747178000
}
branches {
  repo {
    name: "gppd"
    type: "user"
  }
  name: "master"
}
, repo {
  name: "test"
  type: "user"
}
created {
  seconds: 1648809473
  nanos: 914898000
}
]


In [13]:
# Create a file in `(repo="gppd", branch="master")` at `/global_power_plant_database_v_1_3/global_power_plant_database.csv`
with client.commit("gppd", "master") as commit:
    client.put_file_bytes(commit, "/global_power_plant_database_v_1_3/global_power_plant_database.csv", gppd_csv_file)
# Check the file commit
print(list(client.list_file(("gppd","master"), "/global_power_plant_database_v_1_3/")))

[file {
  commit {
    branch {
      repo {
        name: "gppd"
        type: "user"
      }
      name: "master"
    }
    id: "5952c5bd0aa94fbcb60845a364f8c563"
  }
  path: "/global_power_plant_database_v_1_3/global_power_plant_database.csv"
  datum: "default"
}
file_type: FILE
committed {
  seconds: 1648994252
  nanos: 875712000
}
hash: "\016WQ\300&\345C\262\350\253.\260`\231\332\241\321\345\337Gw\217w\207\372\253E\315\361/\343\250"
]


In [14]:
f = client.get_file(("gppd", "master"), "/global_power_plant_database_v_1_3/global_power_plant_database.csv")
# df = pd.read_csv(f)

StopIteration: 

In [ ]:
engine = osc.attach_trino_engine(verbose=True, catalog=ingest_catalog)

In [ ]:
# Show available schemas to ensure trino connection is set correctly
schema_show_sql = f"""
show schemas in {ingest_catalog}
"""
schema_show = engine.execute(schema_show_sql)
print(schema_show.fetchall())